In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Define dataset file paths
datasets = {
    "gas_price": "gas_price.csv",
    "rig_count": "rig_count.csv",
    "tam": "tam.csv",
    "capex": "capex.csv",
    "oil_price": "oil_price.csv",
    "education": "education.csv",
    "political_stability": "political_stability.csv",
    "infrastructure_index": "infrastructure_index.csv",
    "health_index": "health_index.csv",
    "gni": "gni.xlsx",
    "gdp": "gdp.xlsx",
    "population": "population.xlsx",
    "country_mapping": "country_table(in).csv" 
}

def load_data(dataset_name):
    """Loads and processes the dataset."""
    path = datasets.get(dataset_name)
    if not path:
        print(f"Dataset '{dataset_name}' not found in provided paths.")
        return None

    try:
        if path.lower().endswith('.csv'):
            df = pd.read_csv(path)
        elif path.lower().endswith(('.xlsx', '.xls')):
            df = pd.read_excel(path)
        else:
            print(f"Unsupported file format for dataset '{dataset_name}'.")
            return None

        df["country_name"] = df["country_name"].str.strip().str.upper()
        df["year"] = df["year"].astype(int)
        return df
    except Exception as e:
        print(f"Error loading {dataset_name}: {e}")
        return None

        
# ---------- Model Building ----------
def build_nn_model(input_dim):
    """Builds and returns a neural network model."""
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),
        BatchNormalization(),
        Dropout(0.2),
        Dense(32, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# ---------- Training Country-Specific Models (rig_count, tam, capex) ----------
def train_model_for_country(dataset_name, country):
    """Trains a neural network model for a specific country and dataset."""
    df = load_data(dataset_name)
    if df is None:
        print(f"Skipping {country}: Unable to load data for {dataset_name}.")
        return None, None, None, None

    country_df = df[df["country_name"] == country]
    if country_df.empty:
        print(f"Skipping {country}: No data available for {dataset_name}.")
        return None, None, None, None

    # Assume the target feature is the one that is not 'country_name' or 'year'
    target_feature = [col for col in country_df.columns if col in selected_options or col not in ["country_name", "year"]][0]
    scaler = MinMaxScaler(feature_range=(0, 1))

    # Sort and interpolate missing data
    country_df_sorted = country_df.sort_values("year")
    country_df_sorted.interpolate(method="linear", inplace=True)

    X = country_df_sorted.drop(columns=["country_name", "year"])
    y = country_df_sorted[[target_feature]]

    X_scaled = scaler.fit_transform(X)
    y_scaled = scaler.fit_transform(y)

    if len(X_scaled) < 5:
        print(f"Skipping {country}: Not enough data for training in {dataset_name}.")
        return None, None, None, None

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)
    model = build_nn_model(input_dim=X_train.shape[1])

    # Callbacks for training
    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test),
              callbacks=[lr_scheduler, early_stopping], verbose=2)

    return model, scaler, country_df_sorted, target_feature

# ---------- Training Year-Based Models (gas_price, oil_price) ----------
def train_nn_model(df, target_feature):
    """Trains a neural network model using 'year' as the feature."""
    grouped_df = df.groupby("year").mean(numeric_only=True).reset_index()
    X = grouped_df[["year"]].values
    y = grouped_df[[target_feature]].values

    scaler_X, scaler_y = MinMaxScaler(), MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)
    model = build_nn_model(input_dim=1)

    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32,
              callbacks=[lr_scheduler, early_stopping], verbose=2)

    return model, scaler_X, scaler_y

# ---------- Prediction Functions ----------
def predict_future_by_country(model, scaler, df, target_feature, start_year, num_years, country):
    """
    Predicts future values using a country-specific model.
    The returned DataFrame will always use 'year' and 'country_name' as keys.
    If the target is 'rig_count', the prediction is rounded to a whole number.
    """
    if model is None:
        return pd.DataFrame()

    future_predictions = []
    latest_data = df[df["year"] == df["year"].max()]
    latest_scaled = scaler.transform(latest_data.drop(columns=["country_name", "year"]))

    for i in range(num_years):
        predicted_scaled = model.predict(latest_scaled, verbose=0)
        predicted_value = scaler.inverse_transform(predicted_scaled)[0][0]
        trend_factor = np.random.uniform(0.9, 1.1)
        predicted_value *= trend_factor

        if target_feature == "rig_count":
            predicted_value = int(round(predicted_value))

        prediction_year = start_year + i
        future_predictions.append({
            "year": prediction_year,
            "country_name": country,
            target_feature: predicted_value
        })
        print(f"Predicted {target_feature} for {country} in {prediction_year}: {predicted_value}")

    return pd.DataFrame(future_predictions)

def predict_prices(model, scaler_X, scaler_y, start, end, countries, feature_name):
    """
    Predicts prices using a year-based model.
    Returns a DataFrame with 'year' and 'country_name' columns.
    """
    predictions = []
    years = np.arange(start, end + 1).reshape(-1, 1)
    years_scaled = scaler_X.transform(years)
    preds_scaled = model.predict(years_scaled, verbose=0)
    preds = scaler_y.inverse_transform(preds_scaled)

    for country in countries:
        for year, price in zip(years.flatten(), preds.flatten()):
            predictions.append({
                "year": year,
                "country_name": country,
                feature_name: price
            })
    return pd.DataFrame(predictions)
    

# --------------------------------- Main Execution Flow -------------------------------------

# Get dataset names (comma separated)
valid_datasets = {"gas_price", "oil_price", "rig_count", "tam", "capex"}
dataset_names_input = input("\nEnter what you want to predict (gas_price, oil_price, rig_count, tam, capex): ")
dataset_names = []

for name in dataset_names_input.split(','):
    clean_name = name.strip()
    if clean_name in valid_datasets:
        dataset_names.append(clean_name)

for i, dataset_name in enumerate(datasets.keys(), start=1):
    print(f"{i}. {dataset_name}")
# Menu Display (Select Datasets for Prediction)
print("\nAvailable Datasets for Prediction, Here's a catch - If you select all relevant data then predictions is on basis of global indicators else the neural network identify correlation itself adding historical considerations to predict\n")
print("1. Gas Price Prediction: Uses Oil Price, Rig Count, Infrastructure Index, Capex.")
print("2. Rig Count Prediction: Uses Rig Count, Infrastructure Index, Capex, Political Stability.")
print("3. Oil Price Prediction: Uses Rig Count, GDP Growth, Infrastructure Index, Capex, Political Stability.")
print("4. Tam Prediction: Uses Capex, GDP, Population, Health Index, Education, Infrastructure Index.")
print("5. Capex Prediction: Uses Oil Price, Gas Price, Rig Count, GDP Growth, Political Stability, Infrastructure Index.")

selected_options = input("\nEnter the numbers corresponding to the datasets you want to predict: ").strip()

# Load country mapping to get available countries
gas_price_data = load_data("gas_price")
if gas_price_data is not None and "country_name" in gas_price_data.columns:
    available_countries = sorted(gas_price_data["country_name"].unique())
    print("\nAvailable countries:", ", ".join(available_countries))
else:
    print("Gas price data could not be loaded or does not contain country information.")
    available_countries = []


# Get countries from user input
selected_countries_input = input("Enter countries (comma-separated): ")
selected_countries = [c.strip().upper() for c in selected_countries_input.split(',')
                      if c.strip().upper() in available_countries]

# Get prediction year range from user
start_year = int(input("Enter the start year for predictions: "))
end_year = int(input("Enter the end year for predictions: "))

# Train global models for gas and oil prices
oil_price_data = load_data("oil_price")
gas_model, gas_scaler_X, gas_scaler_y = train_nn_model(gas_price_data, "gas_price")
oil_model, oil_scaler_X, oil_scaler_y = train_nn_model(oil_price_data, "oil_price")

# Create a skeleton DataFrame containing all combinations of selected countries and years
years = list(range(start_year, end_year + 1))
skeleton = pd.DataFrame([(year, country) for country in selected_countries for year in years],
                        columns=["year", "country_name"])
final_predictions = skeleton.copy()

# Process each dataset's predictions and merge them into final_predictions
for dataset in dataset_names:
    if dataset in ["gas_price", "oil_price"]:
        # Global models predict for all countries at once.
        if dataset == "gas_price" and gas_model:
            pred_df = predict_prices(gas_model, gas_scaler_X, gas_scaler_y, start_year, end_year,
                                      selected_countries, "gas_price")
        elif dataset == "oil_price" and oil_model:
            pred_df = predict_prices(oil_model, oil_scaler_X, oil_scaler_y, start_year, end_year,
                                      selected_countries, "oil_price")
        else:
            pred_df = pd.DataFrame()
        final_predictions = pd.merge(final_predictions, pred_df, on=["year", "country_name"], how="left")
    elif dataset in ["rig_count", "tam", "capex"]:
        # Country-specific models: train and predict per country.
        all_preds = []
        for country in selected_countries:
            model, scaler, df, target_feature = train_model_for_country(dataset, country)
            if model:
                num_years = end_year - start_year + 1
                pred_df = predict_future_by_country(model, scaler, df, target_feature, start_year, num_years, country)
                all_preds.append(pred_df)
        if all_preds:
            pred_all = pd.concat(all_preds, ignore_index=True)
            final_predictions = pd.merge(final_predictions, pred_all, on=["year", "country_name"], how="left")

def load_country_mapping(file_path):
    """Loads and preprocesses country mappings from the provided CSV file."""
    try:
        country_mapping = pd.read_csv(file_path)
        country_mapping["country_name"] = country_mapping["country_name"].str.strip().str.upper()
        return country_mapping
    except Exception as e:
        print(f"Error loading country mapping: {e}")
        return None

country_mapping = load_country_mapping(datasets["country_mapping"])

if country_mapping is not None and "country_name" in country_mapping.columns and "country_id" in country_mapping.columns:
    final_predictions = pd.merge(final_predictions, country_mapping[["country_name", "country_id"]], on="country_name", how="left")

    # Reorder columns
    columns_order = ["country_id", "country_name", "year"] + [col for col in final_predictions.columns if col not in ["year", "country_name", "country_id"]]
    final_predictions = final_predictions[columns_order]
else:
    print("Country mapping data is invalid or missing required columns.")

# Save predictions
output_file = input("\nEnter the file name to save predictions (e.g., predictions_output.csv): ").strip()
output_file += ".csv" if not output_file.lower().endswith(".csv") else ""

try:
    final_predictions.to_csv(output_file, index=False)
    print(f"\nPredictions have been successfully saved to '{output_file}'.")
except Exception as e:
    print(f"Error saving predictions: {e}")


Enter what you want to predict (gas_price, oil_price, rig_count, tam, capex):  gas_price, oil_price, rig_count, tam, capex


1. gas_price
2. rig_count
3. tam
4. capex
5. oil_price
6. education
7. political_stability
8. infrastructure_index
9. health_index
10. gni
11. gdp
12. population
13. country_mapping

Available Datasets for Prediction, Here's a catch - If you select all relevant data then predictions is on basis of global indicators else the neural network identify correlation itself adding historical considerations to predict

1. Gas Price Prediction: Uses Oil Price, Rig Count, Infrastructure Index, Capex.
2. Rig Count Prediction: Uses Rig Count, Infrastructure Index, Capex, Political Stability.
3. Oil Price Prediction: Uses Rig Count, GDP Growth, Infrastructure Index, Capex, Political Stability.
4. Tam Prediction: Uses Capex, GDP, Population, Health Index, Education, Infrastructure Index.
5. Capex Prediction: Uses Oil Price, Gas Price, Rig Count, GDP Growth, Political Stability, Infrastructure Index.



Enter the numbers corresponding to the datasets you want to predict:  



Available countries: AFGHANISTAN, ALBANIA, ALGERIA, ANDORRA, ANGOLA, ANTIGUA & BARBUDA, ARGENTINA, ARMENIA, AUSTRALIA, AUSTRIA, AZERBAIJAN, BAHRAIN, BANGLADESH, BELARUS, BELGIUM, BELIZE, BENIN, BHUTAN, BOLIVIA, BOSNIA & HERZEGOVINA, BOTSWANA, BRAZIL, BRUNEI DARUSSALAM, BULGARIA, BURKINA FASO, BURUNDI, CAMBODIA, CAMEROON, CANADA, CHAD, CHILE, CHINA, COLOMBIA, CONGO, DEM. REP., COSTA RICA, COTE D'IVOIRE, CROATIA, CUBA, CYPRUS, CZECH REPUBLIC, DENMARK, DJIBOUTI, DOMINICAN REPUBLIC, ECUADOR, EGYPT, EL SALVADOR, EQUATORIAL GUINEA, ESTONIA, ETHIOPIA, FINLAND, FRANCE, GAMBIA, GEORGIA, GERMANY, GHANA, GREECE, GUATEMALA, GUINEA, GUINEA-BISSAU, GUYANA, HAITI, HONDURAS, HUNGARY, ICELAND, INDIA, INDONESIA, IRAN, IRAQ, IRELAND, ISRAEL, ITALY, JAMAICA, JAPAN, JORDAN, KAZAKHSTAN, KENYA, KIRIBATI, KUWAIT, KYRGYZ REPUBLIC, LAOS, LATVIA, LEBANON, LESOTHO, LIBERIA, LIBYA, LITHUANIA, LUXEMBOURG, MADAGASCAR, MALAWI, MALAYSIA, MALDIVES, MALI, MALTA, MAURITANIA, MAURITIUS, MEXICO, MICRONESIA, MOLDOVA, MONAC

Enter countries (comma-separated):  Afghanistan, Albania, angola
Enter the start year for predictions:  2026
Enter the end year for predictions:  2030


...

1/1 - 3s - 3s/step - loss: 1.2585 - mae: 0.7945 - val_loss: 0.0842 - val_mae: 0.2224 - learning_rate: 0.0010
Epoch 2/50
1/1 - 0s - 94ms/step - loss: 0.9265 - mae: 0.7035 - val_loss: 0.0790 - val_mae: 0.2147 - learning_rate: 0.0010
Epoch 3/50
1/1 - 0s - 93ms/step - loss: 2.3499 - mae: 1.1080 - val_loss: 0.0747 - val_mae: 0.2087 - learning_rate: 0.0010
Epoch 4/50
1/1 - 0s - 91ms/step - loss: 2.1135 - mae: 0.8104 - val_loss: 0.0698 - val_mae: 0.2020 - learning_rate: 0.0010
Epoch 5/50
1/1 - 0s - 113ms/step - loss: 0.6111 - mae: 0.5334 - val_loss: 0.0655 - val_mae: 0.1969 - learning_rate: 0.0010
Epoch 6/50
1/1 - 0s - 91ms/step - loss: 0.7470 - mae: 0.6812 - val_loss: 0.0615 - val_mae: 0.1917 - learning_rate: 0.0010
Epoch 7/50
1/1 - 0s - 108ms/step - loss: 1.6094 - mae: 0.9432 - val_loss: 0.0578 - val_mae: 0.1868 - learning_rate: 0.0010
Epoch 8/50
1/1 - 0s - 105ms/step - loss: 2.0852 - mae: 0.8987 - val_loss: 0.0544 - val_mae: 0.1824 - learning_rate: 0.0010
Epoch 9/50
1/1 - 0s - 107ms/step -

C:\Users\asus\AppData\Local\Temp\ipykernel_18160\3858047369.py:85: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  country_df_sorted.interpolate(method="linear", inplace=True)
C:\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 - 3s - 3s/step - loss: 4.1011 - mae: 1.7780 - val_loss: 0.1003 - val_mae: 0.2315 - learning_rate: 0.0010
Epoch 2/50
1/1 - 0s - 110ms/step - loss: 4.9467 - mae: 1.8507 - val_loss: 0.1041 - val_mae: 0.2361 - learning_rate: 0.0010
Epoch 3/50
1/1 - 0s - 81ms/step - loss: 1.7830 - mae: 1.2059 - val_loss: 0.1049 - val_mae: 0.2365 - learning_rate: 0.0010
Epoch 4/50
1/1 - 0s - 99ms/step - loss: 2.1387 - mae: 1.1758 - val_loss: 0.1057 - val_mae: 0.2383 - learning_rate: 0.0010
Epoch 5/50
1/1 - 0s - 86ms/step - loss: 3.6845 - mae: 1.5835 - val_loss: 0.1034 - val_mae: 0.2386 - learning_rate: 0.0010
Epoch 6/50

Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
1/1 - 0s - 116ms/step - loss: 2.5883 - mae: 1.3283 - val_loss: 0.1019 - val_mae: 0.2390 - learning_rate: 0.0010
Epoch 7/50
1/1 - 0s - 87ms/step - loss: 3.2570 - mae: 1.5003 - val_loss: 0.1021 - val_mae: 0.2396 - learning_rate: 5.0000e-04
Epoch 8/50
1/1 - 0s - 112ms/step - loss: 2.3070 - mae: 1.3516 - val_loss: 0.


Enter the file name to save predictions (e.g., predictions_output.csv):  cl2



Predictions have been successfully saved to 'cl2.csv'.


In [3]:
import os
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.initializers import GlorotUniform
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Fix random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)
os.environ['PYTHONHASHSEED'] = '42'

# Dataset paths
datasets = {
    "gas_price": "gas_price.csv",
    "rig_count": "rig_count.csv",
    "tam": "tam.csv",
    "capex": "capex.csv",
    "oil_price": "oil_price.csv",
    "education": "education.csv",
    "political_stability": "political_stability.csv",
    "infrastructure_index": "infrastructure_index.csv",
    "health_index": "health_index.csv",
    "gni": "gni.xlsx",
    "gdp": "gdp.xlsx",
    "population": "population.xlsx",
    "country_mapping": "country_table(in).csv" 
}

def load_data(dataset_name):
    try:
        df = pd.read_csv(datasets[dataset_name]) if datasets[dataset_name].endswith('.csv') else pd.read_excel(datasets[dataset_name])
        df["country_name"] = df["country_name"].str.strip().str.upper()
        df["year"] = df["year"].astype(int)
        return df
    except Exception as e:
        print(f"Error loading {dataset_name}: {e}")
        return None

def build_nn_model(input_dim):
    initializer = GlorotUniform(seed=42)
    model = Sequential([
        Dense(64, activation='relu', kernel_initializer=initializer, input_shape=(input_dim,)),
        BatchNormalization(),
        Dropout(0.2, seed=42),
        Dense(32, activation='relu', kernel_initializer=initializer),
        BatchNormalization(),
        Dropout(0.2, seed=42),
        Dense(1, activation='linear', kernel_initializer=initializer)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# ---------- Training Country-Specific Models (rig_count, tam, capex) ----------
def train_model_for_country(dataset_name, country):
    """Trains a neural network model for a specific country and dataset."""
    df = load_data(dataset_name)
    if df is None:
        print(f"Skipping {country}: Unable to load data for {dataset_name}.")
        return None, None, None, None

    country_df = df[df["country_name"] == country]
    if country_df.empty:
        print(f"Skipping {country}: No data available for {dataset_name}.")
        return None, None, None, None

    # Assume the target feature is the one that is not 'country_name' or 'year'
    target_feature = [col for col in country_df.columns if col in selected_options or col not in ["country_name", "year"]][0]
    scaler = MinMaxScaler(feature_range=(0, 1))

    # Sort and interpolate missing data
    country_df_sorted = country_df.sort_values("year")
    country_df_sorted.interpolate(method="linear", inplace=True)

    X = country_df_sorted.drop(columns=["country_name", "year"])
    y = country_df_sorted[[target_feature]]

    X_scaled = scaler.fit_transform(X)
    y_scaled = scaler.fit_transform(y)

    if len(X_scaled) < 5:
        print(f"Skipping {country}: Not enough data for training in {dataset_name}.")
        return None, None, None, None

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)
    model = build_nn_model(input_dim=X_train.shape[1])

    # Callbacks for training
    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test),
              callbacks=[lr_scheduler, early_stopping], verbose=2)

    return model, scaler, country_df_sorted, target_feature

# ---------- Training Year-Based Models (gas_price, oil_price) ----------
def train_nn_model(df, target_feature):
    """Trains a neural network model using 'year' as the feature."""
    grouped_df = df.groupby("year").mean(numeric_only=True).reset_index()
    X = grouped_df[["year"]].values
    y = grouped_df[[target_feature]].values

    scaler_X, scaler_y = MinMaxScaler(), MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)
    model = build_nn_model(input_dim=1)

    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32,
              callbacks=[lr_scheduler, early_stopping], verbose=2)

    return model, scaler_X, scaler_y

# ---------- Prediction Functions ----------
def predict_future_by_country(model, scaler, df, target_feature, start_year, num_years, country):
    """
    Predicts future values using a country-specific model.
    The returned DataFrame will always use 'year' and 'country_name' as keys.
    If the target is 'rig_count', the prediction is rounded to a whole number.
    """
    if model is None:
        return pd.DataFrame()

    future_predictions = []
    latest_data = df[df["year"] == df["year"].max()]
    latest_scaled = scaler.transform(latest_data.drop(columns=["country_name", "year"]))

    for i in range(num_years):
        predicted_scaled = model.predict(latest_scaled, verbose=0)
        predicted_value = scaler.inverse_transform(predicted_scaled)[0][0]
        trend_factor = np.random.uniform(0.9, 1.1)
        predicted_value *= trend_factor

        if target_feature == "rig_count":
            predicted_value = int(round(predicted_value))

        prediction_year = start_year + i
        future_predictions.append({
            "year": prediction_year,
            "country_name": country,
            target_feature: predicted_value
        })
        print(f"Predicted {target_feature} for {country} in {prediction_year}: {predicted_value}")

    return pd.DataFrame(future_predictions)

def predict_prices(model, scaler_X, scaler_y, start, end, countries, feature_name):
    """
    Predicts prices using a year-based model.
    Returns a DataFrame with 'year' and 'country_name' columns.
    """
    predictions = []
    years = np.arange(start, end + 1).reshape(-1, 1)
    years_scaled = scaler_X.transform(years)
    preds_scaled = model.predict(years_scaled, verbose=0)
    preds = scaler_y.inverse_transform(preds_scaled)

    for country in countries:
        for year, price in zip(years.flatten(), preds.flatten()):
            predictions.append({
                "year": year,
                "country_name": country,
                feature_name: price
            })
    return pd.DataFrame(predictions)
    

# --------------------------------- Main Execution Flow -------------------------------------

# Get dataset names (comma separated)
valid_datasets = {"gas_price", "oil_price", "rig_count", "tam", "capex"}
dataset_names_input = input("\nEnter what you want to predict (gas_price, oil_price, rig_count, tam, capex): ")
dataset_names = []

for name in dataset_names_input.split(','):
    clean_name = name.strip()
    if clean_name in valid_datasets:
        dataset_names.append(clean_name)

for i, dataset_name in enumerate(datasets.keys(), start=1):
    print(f"{i}. {dataset_name}")
# Menu Display (Select Datasets for Prediction)
print("\nAvailable Datasets for Prediction, Here's a catch - If you select all relevant data then predictions is on basis of global indicators else the neural network identify correlation itself adding historical considerations to predict\n")
print("1. Gas Price Prediction: Uses Oil Price, Rig Count, Infrastructure Index, Capex.")
print("2. Rig Count Prediction: Uses Rig Count, Infrastructure Index, Capex, Political Stability.")
print("3. Oil Price Prediction: Uses Rig Count, GDP Growth, Infrastructure Index, Capex, Political Stability.")
print("4. Tam Prediction: Uses Capex, GDP, Population, Health Index, Education, Infrastructure Index.")
print("5. Capex Prediction: Uses Oil Price, Gas Price, Rig Count, GDP Growth, Political Stability, Infrastructure Index.")

selected_options = input("\nEnter the numbers corresponding to the datasets you want to predict: ").strip()

# Load country mapping to get available countries
gas_price_data = load_data("gas_price")
if gas_price_data is not None and "country_name" in gas_price_data.columns:
    available_countries = sorted(gas_price_data["country_name"].unique())
    print("\nAvailable countries:", ", ".join(available_countries))
else:
    print("Gas price data could not be loaded or does not contain country information.")
    available_countries = []


# Get countries from user input
selected_countries_input = input("Enter countries (comma-separated): ")
selected_countries = [c.strip().upper() for c in selected_countries_input.split(',')
                      if c.strip().upper() in available_countries]

# Get prediction year range from user
start_year = int(input("Enter the start year for predictions: "))
end_year = int(input("Enter the end year for predictions: "))

# Train global models for gas and oil prices
oil_price_data = load_data("oil_price")
gas_model, gas_scaler_X, gas_scaler_y = train_nn_model(gas_price_data, "gas_price")
oil_model, oil_scaler_X, oil_scaler_y = train_nn_model(oil_price_data, "oil_price")

# Create a skeleton DataFrame containing all combinations of selected countries and years
years = list(range(start_year, end_year + 1))
skeleton = pd.DataFrame([(year, country) for country in selected_countries for year in years],
                        columns=["year", "country_name"])
final_predictions = skeleton.copy()

# Process each dataset's predictions and merge them into final_predictions
for dataset in dataset_names:
    if dataset in ["gas_price", "oil_price"]:
        # Global models predict for all countries at once.
        if dataset == "gas_price" and gas_model:
            pred_df = predict_prices(gas_model, gas_scaler_X, gas_scaler_y, start_year, end_year,
                                      selected_countries, "gas_price")
        elif dataset == "oil_price" and oil_model:
            pred_df = predict_prices(oil_model, oil_scaler_X, oil_scaler_y, start_year, end_year,
                                      selected_countries, "oil_price")
        else:
            pred_df = pd.DataFrame()
        final_predictions = pd.merge(final_predictions, pred_df, on=["year", "country_name"], how="left")
    elif dataset in ["rig_count", "tam", "capex"]:
        # Country-specific models: train and predict per country.
        all_preds = []
        for country in selected_countries:
            model, scaler, df, target_feature = train_model_for_country(dataset, country)
            if model:
                num_years = end_year - start_year + 1
                pred_df = predict_future_by_country(model, scaler, df, target_feature, start_year, num_years, country)
                all_preds.append(pred_df)
        if all_preds:
            pred_all = pd.concat(all_preds, ignore_index=True)
            final_predictions = pd.merge(final_predictions, pred_all, on=["year", "country_name"], how="left")

def load_country_mapping(file_path):
    """Loads and preprocesses country mappings from the provided CSV file."""
    try:
        country_mapping = pd.read_csv(file_path)
        country_mapping["country_name"] = country_mapping["country_name"].str.strip().str.upper()
        return country_mapping
    except Exception as e:
        print(f"Error loading country mapping: {e}")
        return None

country_mapping = load_country_mapping(datasets["country_mapping"])

if country_mapping is not None and "country_name" in country_mapping.columns and "country_id" in country_mapping.columns:
    final_predictions = pd.merge(final_predictions, country_mapping[["country_name", "country_id"]], on="country_name", how="left")

    # Reorder columns
    columns_order = ["country_id", "country_name", "year"] + [col for col in final_predictions.columns if col not in ["year", "country_name", "country_id"]]
    final_predictions = final_predictions[columns_order]
else:
    print("Country mapping data is invalid or missing required columns.")

# Save predictions
output_file = input("\nEnter the file name to save predictions (e.g., predictions_output.csv): ").strip()
output_file += ".csv" if not output_file.lower().endswith(".csv") else ""

try:
    final_predictions.to_csv(output_file, index=False)
    print(f"\nPredictions have been successfully saved to '{output_file}'.")
except Exception as e:
    print(f"Error saving predictions: {e}")


Enter what you want to predict (gas_price, oil_price, rig_count, tam, capex):  gas_price, oil_price, rig_count, tam, capex


1. gas_price
2. rig_count
3. tam
4. capex
5. oil_price
6. education
7. political_stability
8. infrastructure_index
9. health_index
10. gni
11. gdp
12. population
13. country_mapping

Available Datasets for Prediction, Here's a catch - If you select all relevant data then predictions is on basis of global indicators else the neural network identify correlation itself adding historical considerations to predict

1. Gas Price Prediction: Uses Oil Price, Rig Count, Infrastructure Index, Capex.
2. Rig Count Prediction: Uses Rig Count, Infrastructure Index, Capex, Political Stability.
3. Oil Price Prediction: Uses Rig Count, GDP Growth, Infrastructure Index, Capex, Political Stability.
4. Tam Prediction: Uses Capex, GDP, Population, Health Index, Education, Infrastructure Index.
5. Capex Prediction: Uses Oil Price, Gas Price, Rig Count, GDP Growth, Political Stability, Infrastructure Index.



Enter the numbers corresponding to the datasets you want to predict:  



Available countries: AFGHANISTAN, ALBANIA, ALGERIA, ANDORRA, ANGOLA, ANTIGUA & BARBUDA, ARGENTINA, ARMENIA, AUSTRALIA, AUSTRIA, AZERBAIJAN, BAHRAIN, BANGLADESH, BELARUS, BELGIUM, BELIZE, BENIN, BHUTAN, BOLIVIA, BOSNIA & HERZEGOVINA, BOTSWANA, BRAZIL, BRUNEI DARUSSALAM, BULGARIA, BURKINA FASO, BURUNDI, CAMBODIA, CAMEROON, CANADA, CHAD, CHILE, CHINA, COLOMBIA, CONGO, DEM. REP., COSTA RICA, COTE D'IVOIRE, CROATIA, CUBA, CYPRUS, CZECH REPUBLIC, DENMARK, DJIBOUTI, DOMINICAN REPUBLIC, ECUADOR, EGYPT, EL SALVADOR, EQUATORIAL GUINEA, ESTONIA, ETHIOPIA, FINLAND, FRANCE, GAMBIA, GEORGIA, GERMANY, GHANA, GREECE, GUATEMALA, GUINEA, GUINEA-BISSAU, GUYANA, HAITI, HONDURAS, HUNGARY, ICELAND, INDIA, INDONESIA, IRAN, IRAQ, IRELAND, ISRAEL, ITALY, JAMAICA, JAPAN, JORDAN, KAZAKHSTAN, KENYA, KIRIBATI, KUWAIT, KYRGYZ REPUBLIC, LAOS, LATVIA, LEBANON, LESOTHO, LIBERIA, LIBYA, LITHUANIA, LUXEMBOURG, MADAGASCAR, MALAWI, MALAYSIA, MALDIVES, MALI, MALTA, MAURITANIA, MAURITIUS, MEXICO, MICRONESIA, MOLDOVA, MONAC

Enter countries (comma-separated):  Afghanistan, Albania, angola
Enter the start year for predictions:  2026
Enter the end year for predictions:  2030


...

1/1 - 2s - 2s/step - loss: 1.0624 - mae: 0.8013 - val_loss: 0.0929 - val_mae: 0.2342 - learning_rate: 0.0010
Epoch 2/50
1/1 - 0s - 105ms/step - loss: 1.3992 - mae: 0.8555 - val_loss: 0.0902 - val_mae: 0.2306 - learning_rate: 0.0010
Epoch 3/50
1/1 - 0s - 107ms/step - loss: 3.0339 - mae: 1.1163 - val_loss: 0.0882 - val_mae: 0.2280 - learning_rate: 0.0010
Epoch 4/50
1/1 - 0s - 92ms/step - loss: 2.3358 - mae: 1.0430 - val_loss: 0.0862 - val_mae: 0.2253 - learning_rate: 0.0010
Epoch 5/50
1/1 - 0s - 109ms/step - loss: 1.2981 - mae: 0.8178 - val_loss: 0.0841 - val_mae: 0.2227 - learning_rate: 0.0010
Epoch 6/50
1/1 - 0s - 86ms/step - loss: 1.4699 - mae: 0.9224 - val_loss: 0.0813 - val_mae: 0.2204 - learning_rate: 0.0010
Epoch 7/50
1/1 - 0s - 91ms/step - loss: 2.2336 - mae: 1.1298 - val_loss: 0.0787 - val_mae: 0.2185 - learning_rate: 0.0010
Epoch 8/50
1/1 - 0s - 108ms/step - loss: 1.4861 - mae: 0.8365 - val_loss: 0.0764 - val_mae: 0.2164 - learning_rate: 0.0010
Epoch 9/50
1/1 - 0s - 104ms/step 

C:\Users\asus\AppData\Local\Temp\ipykernel_18160\1022130620.py:79: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  country_df_sorted.interpolate(method="linear", inplace=True)
C:\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 - 3s - 3s/step - loss: 1.3622 - mae: 0.7887 - val_loss: 0.2291 - val_mae: 0.3473 - learning_rate: 0.0010
Epoch 2/50
1/1 - 0s - 105ms/step - loss: 1.2572 - mae: 0.8485 - val_loss: 0.2240 - val_mae: 0.3429 - learning_rate: 0.0010
Epoch 3/50
1/1 - 0s - 104ms/step - loss: 1.2883 - mae: 0.9074 - val_loss: 0.2177 - val_mae: 0.3377 - learning_rate: 0.0010
Epoch 4/50
1/1 - 0s - 82ms/step - loss: 2.3099 - mae: 0.9635 - val_loss: 0.2116 - val_mae: 0.3326 - learning_rate: 0.0010
Epoch 5/50
1/1 - 0s - 97ms/step - loss: 1.0588 - mae: 0.6826 - val_loss: 0.2055 - val_mae: 0.3287 - learning_rate: 0.0010
Epoch 6/50
1/1 - 0s - 92ms/step - loss: 1.2092 - mae: 0.8271 - val_loss: 0.1997 - val_mae: 0.3250 - learning_rate: 0.0010
Epoch 7/50
1/1 - 0s - 100ms/step - loss: 2.1648 - mae: 1.0948 - val_loss: 0.1959 - val_mae: 0.3224 - learning_rate: 0.0010
Epoch 8/50
1/1 - 0s - 94ms/step - loss: 1.4261 - mae: 1.0606 - val_loss: 0.1936 - val_mae: 0.3207 - learning_rate: 0.0010
Epoch 9/50
1/1 - 0s - 113ms/step -


Enter the file name to save predictions (e.g., predictions_output.csv):  cl1



Predictions have been successfully saved to 'cl1.csv'.
